# In this file, dummy csv files(sheets) are compared using pandas, dataframe and datasets.

# Dask is a library used to check the parallel computing and the time taken by files to process the computing.

In [1]:
import pandas as pd
import numpy as np
from dask import dataframe as dd
import time
import os
from dask.distributed import Client

# This module provides an interface to the optional garbage collector. It provides the ability to disable the collector, tune the collection frequency, and set debugging options.

In [2]:
# Just in case, to clean up RAM
import gc
gc.collect()

350

In [7]:
df.shape  #--> gives number of rows:cols

(9, 3)

# head () function is used to access the first n rows of a dataframe or series.

# Accordingly tail() returns the last n rows.

In [8]:
df.head()

,Tran_Id,TranDate,TranAmnt
0,123,09-02-2022,200000
1,345,09-02-2022,300000
2,956,09-02-2022,500000
3,524,09-02-2022,800000
4,654,09-02-2022,200000


In [4]:
#SIZE OF DATA TO BE IMPORTED
print(os.path.getsize(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask1.csv')/1024/1024/1024 , "GB")

2.1792948246002197e-07 GB


In [5]:
start = time.time()
df = pd.read_csv(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask1.csv')
end = time.time()
print("Read csv without chunks: ",(end-start),"sec")

Read csv without chunks:  0.10313987731933594 sec


In [6]:
start = time.time()
chunk = pd.read_csv(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask1.csv',chunksize=1000000)
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")
pd_df = pd.concat(chunk)

Read csv with chunks:  0.0009949207305908203 sec


In [9]:
type(chunk)

pandas.io.parsers.TextFileReader

In [ ]:
#The Client registers itself as the default Dask scheduler, and so runs all dask collections like dask.array, dask.bag, dask.dataframe and dask.delayed

In [10]:
client = Client(processes=False)
client

C:\Users\Dell\anaconda3\lib\site-packages\distributed\node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 64980 instead
  warnings.warn(


Client Scheduler: inproc://192.168.0.109/5448/1 Dashboard: http://192.168.0.109:64980/status,Cluster Workers: 1 Cores: 4 Memory: 7.90 GiB


In [37]:
start = time.time()
dask_df = dd.read_csv(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask1.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")
pd_df = dask_df.compute()
print("Dask size:", dask_df.shape)

Read csv with dask:  0.01399683952331543 sec
Dask size: (Delayed('int-ec8ad790-5b96-4e1c-8201-6f2f055dd3be'), 3)


In [12]:
df1 = pd.read_csv(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask1.csv')
print (df1)

   Tran_Id    TranDate  TranAmnt
0      123  09-02-2022    200000
1      345  09-02-2022    300000
2      956  09-02-2022    500000
3      524  09-02-2022    800000
4      654  09-02-2022    200000
5      123  09-02-2022    200000
6      345  09-02-2022    300000
7      654  09-02-2022    500000
8      123  15-02-2022    100000


In [11]:
df2 = pd.read_csv(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask2.csv')
print(df2)

   Tran_Id    TranDate  TranAmnt
0      123  09-02-2022    200000
1      345  09-02-2022    260000
2      400  15-02-2022    500000
3      524  09-02-2022    800000
4      654  09-02-2022    200000
5      500  09-02-2022    260000
6      999  09-02-2022    300000
7      210  09-02-2022    700000
8      874  15-02-2022    100000


In [46]:
pd.merge(df1,df2 , on = 'Tran_Id', how = 'inner')

,Tran_Id,TranDate_x,TranAmnt_x,TranDate_y,TranAmnt_y
0,123,09-02-2022,200000,09-02-2022,200000
1,123,09-02-2022,200000,09-02-2022,200000
2,123,15-02-2022,100000,09-02-2022,200000
3,345,09-02-2022,300000,09-02-2022,260000
4,345,09-02-2022,300000,09-02-2022,260000
5,524,09-02-2022,800000,09-02-2022,800000
6,654,09-02-2022,200000,09-02-2022,200000
7,654,09-02-2022,500000,09-02-2022,200000


In [ ]:
#The isin() method checks if the Dataframe contains the specified value(s).

In [13]:
df1.apply(tuple,1).isin(df2.apply(tuple,1))

0     True
1    False
2    False
3     True
4     True
5     True
6    False
7    False
8    False
dtype: bool

In [14]:
start = time.time()
result = df1[df1.apply(tuple,1).isin(df2.apply(tuple,1))]
end = time.time()
print (result, '\n',(end-start),'sec')

   Tran_Id    TranDate  TranAmnt
0      123  09-02-2022    200000
3      524  09-02-2022    800000
4      654  09-02-2022    200000
5      123  09-02-2022    200000 
 0.006981849670410156 sec


In [61]:
start = time.time()
result2 = df1.merge(df2, how= 'inner', indicator = True)
end = time.time()
print (result2,'\n',(end-start),'sec')

   Tran_Id    TranDate  TranAmnt _merge
0      123  09-02-2022    200000   both
1      123  09-02-2022    200000   both
2      524  09-02-2022    800000   both
3      654  09-02-2022    200000   both 
 0.021940231323242188 sec


In [62]:
start = time.time()
result3 = df1.merge(df2, how= 'outer', indicator = True).loc[lambda v :v['_merge'] != 'both']
end = time.time()
print (result3,'\n',(end-start),'sec')

    Tran_Id    TranDate  TranAmnt      _merge
2       345  09-02-2022    300000   left_only
3       345  09-02-2022    300000   left_only
4       956  09-02-2022    500000   left_only
7       654  09-02-2022    500000   left_only
8       123  15-02-2022    100000   left_only
9       345  09-02-2022    260000  right_only
10      400  15-02-2022    500000  right_only
11      500  09-02-2022    260000  right_only
12      999  09-02-2022    300000  right_only
13      210  09-02-2022    700000  right_only
14      874  15-02-2022    100000  right_only 
 0.055845022201538086 sec


In [ ]:
#This returns all rows from both tables, join records from the left which have matching keys in the right table.When there is no Matching from any table NaN will be returned

In [87]:
start = time.time()
data_merge = df1.merge(df2, how = 'outer', left_on = "Tran_Id", right_on = "Tran_Id")
end = time.time()
print (data_merge,'\n',(end-start),'sec')


    Tran_Id  TranDate_x  TranAmnt_x  TranDate_y  TranAmnt_y
0       123  09-02-2022    200000.0  09-02-2022    200000.0
1       123  09-02-2022    200000.0  09-02-2022    200000.0
2       123  15-02-2022    100000.0  09-02-2022    200000.0
3       345  09-02-2022    300000.0  09-02-2022    260000.0
4       345  09-02-2022    300000.0  09-02-2022    260000.0
5       956  09-02-2022    500000.0         NaN         NaN
6       524  09-02-2022    800000.0  09-02-2022    800000.0
7       654  09-02-2022    200000.0  09-02-2022    200000.0
8       654  09-02-2022    500000.0  09-02-2022    200000.0
9       400         NaN         NaN  15-02-2022    500000.0
10      500         NaN         NaN  09-02-2022    260000.0
11      999         NaN         NaN  09-02-2022    300000.0
12      210         NaN         NaN  09-02-2022    700000.0
13      874         NaN         NaN  15-02-2022    100000.0 
 0.008977174758911133 sec


In [88]:
start = time.time()
data_merge2 = df1.merge(df2, how = 'inner', left_on = "TranAmnt", right_on = "TranAmnt")
end = time.time()
print (data_merge2,'\n',(end-start),'sec')


    Tran_Id_x  TranDate_x  TranAmnt  Tran_Id_y  TranDate_y
0         123  09-02-2022    200000        123  09-02-2022
1         123  09-02-2022    200000        654  09-02-2022
2         654  09-02-2022    200000        123  09-02-2022
3         654  09-02-2022    200000        654  09-02-2022
4         123  09-02-2022    200000        123  09-02-2022
5         123  09-02-2022    200000        654  09-02-2022
6         345  09-02-2022    300000        999  09-02-2022
7         345  09-02-2022    300000        999  09-02-2022
8         956  09-02-2022    500000        400  15-02-2022
9         654  09-02-2022    500000        400  15-02-2022
10        524  09-02-2022    800000        524  09-02-2022
11        123  15-02-2022    100000        874  15-02-2022 
 0.009974002838134766 sec


In [78]:
#compare files
with open(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask1.csv') as t1, open(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask2.csv') as t2:
    fileone = t1.readlines()
    filetwo = t2.readlines()

with open('update.csv','w') as outFile:
    for line in filetwo:
        if line not in fileone:
            outFile.write(line)
            end = time.time()

#update.csv will give difference of both files which is compared.

In [89]:
start = time.time()
result3 = pd.merge(df1,df2, on= ['Tran_Id', 'TranAmnt'], how = 'inner')
end = time.time()
print (result3,'\n',(end-start),'sec')

   Tran_Id  TranDate_x  TranAmnt  TranDate_y
0      123  09-02-2022    200000  09-02-2022
1      123  09-02-2022    200000  09-02-2022
2      524  09-02-2022    800000  09-02-2022
3      654  09-02-2022    200000  09-02-2022 
 0.00697779655456543 sec


In [92]:
start = time.time()
result5 = pd.merge(df1,df2, on= ['TranDate', 'TranAmnt', 'Tran_Id'], how = 'inner')
end = time.time()
print (result5,'\n',(end-start),'sec')

   Tran_Id    TranDate  TranAmnt
0      123  09-02-2022    200000
1      123  09-02-2022    200000
2      524  09-02-2022    800000
3      654  09-02-2022    200000 
 0.028254985809326172 sec


# Computing comparisn based on the files received. 

In [6]:
start = time.time()
dask_df1 = dd.read_csv(r'C:\Users\Dell\Desktop\New folder\Test_1\Test\df1_out_FICL.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")
pd_df = dask_df1.compute()
print("Dask size:", dask_df1.shape)

Read csv with dask:  0.012938976287841797 sec
Dask size: (Delayed('int-536241db-08ce-4e43-ab94-272ea924e6e3'), 7)


In [7]:
start = time.time()
dask_df2 = dd.read_csv(r'C:\Users\Dell\Desktop\New folder\Test_1\Test\df2_out_SCLM.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")
pd_df = dask_df2.compute()
print("Dask size:", dask_df2.shape)

Read csv with dask:  0.01695704460144043 sec
Dask size: (Delayed('int-07c685c7-a67f-401b-984c-365d48213831'), 7)


In [9]:
dataf1 = pd.read_csv(r'C:\Users\Dell\Desktop\New folder\Test_1\Test\df1_out_FICL.csv')
print (dataf1.head())

   Unnamed: 0 TRAN_DATE    TRAN_ID               TRAN_REMARKS     ACCOUNTNO  \
0           0  9/2/2022  S85305249                        NaN  0103SLMPSWDL   
1           1  9/2/2022  S85305249                        NaN  0103SLMPSWDL   
2           2  9/2/2022  S88796439  224500032693//10101170199  0103SLMPSWDL   
3           3  9/2/2022  S80939874                        NaN  0103SLMPSWDL   
4           4  9/2/2022  S80939874                        NaN  0103SLMPSWDL   

     TRANAMOUNT           REFERENCENO  
0 -3.192820e+08  OTHERS-P SETT DT01.0  
1 -4.042220e+03  OTHERS-P DIFF DT 31.  
2 -2.774900e+03                   NaN  
3 -6.637716e+06                     -  
4 -7.248358e+05                     -  


In [10]:
dataf2 = pd.read_csv(r'C:\Users\Dell\Desktop\New folder\Test_1\Test\df2_out_SCLM.csv')
print (dataf2.head())

   Unnamed: 0   TRAN_DATE    TRANAMOUNT     ACCOUNTNO           REFERENCENO  \
0           0  09-02-2022  3.192922e+08  0103SLMPSWDL  OTHERS-P SETT DT 01.   
1           1  09-02-2022  1.565600e+02  0103SLMPSWDL  OTHERS-PSETTL DT 01.   
2           2  09-02-2022  2.000000e+00  0103SLMPSWDL  OTHERS-PLPR - DT 01.   
3           3  09-02-2022  4.872000e+05  0103SLMPSWDL  OTHERS-PMDU - DT 01.   
4           4  09-02-2022  1.777348e+07  0103SLMPSWDL   OTHERS-PCI POS SETT   

  TRAN_REMARKS    TRAN_ID  
0          NaN  S85305249  
1          NaN  S85305249  
2          NaN  S85305249  
3          NaN  S85305249  
4          NaN  S85321543  


In [11]:
dataf1.apply(tuple,1).isin(dataf2.apply(tuple,1))

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
dtype: bool

In [12]:
start = time.time()
result = dataf1[dataf1.apply(tuple,1).isin(dataf2.apply(tuple,1))]
end = time.time()
print (result, '\n',(end-start),'sec')

Empty DataFrame
Columns: [Unnamed: 0, TRAN_DATE, TRAN_ID, TRAN_REMARKS, ACCOUNTNO, TRANAMOUNT, REFERENCENO]
Index: [] 
 0.0059854984283447266 sec


In [15]:
start = time.time()
result2 = dataf1.merge(dataf2, how= 'inner', indicator = True)
end = time.time()
print (result2,'\n',(end-start),'sec')

Empty DataFrame
Columns: [Unnamed: 0, TRAN_DATE, TRAN_ID, TRAN_REMARKS, ACCOUNTNO, TRANAMOUNT, REFERENCENO, _merge]
Index: [] 
 0.02991938591003418 sec


# Merged data on TRAN_ID

In [65]:
start = time.time()
data_merge = dataf1.merge(dataf2, how = 'inner', left_on = "TRAN_ID", right_on = "TRAN_ID")
end = time.time()
print (data_merge.head(10),'\n',(end-start),'sec')

   Unnamed: 0_x TRAN_DATE_x    TRAN_ID TRAN_REMARKS_x   ACCOUNTNO_x  \
0             0    9/2/2022  S85305249            NaN  0103SLMPSWDL   
1             0    9/2/2022  S85305249            NaN  0103SLMPSWDL   
2             0    9/2/2022  S85305249            NaN  0103SLMPSWDL   
3             0    9/2/2022  S85305249            NaN  0103SLMPSWDL   
4             0    9/2/2022  S85305249            NaN  0103SLMPSWDL   
5             1    9/2/2022  S85305249            NaN  0103SLMPSWDL   
6             1    9/2/2022  S85305249            NaN  0103SLMPSWDL   
7             1    9/2/2022  S85305249            NaN  0103SLMPSWDL   
8             1    9/2/2022  S85305249            NaN  0103SLMPSWDL   
9             1    9/2/2022  S85305249            NaN  0103SLMPSWDL   

   TRANAMOUNT_x         REFERENCENO_x  Unnamed: 0_y TRAN_DATE_y  TRANAMOUNT_y  \
0 -3.192820e+08  OTHERS-P SETT DT01.0             0  09-02-2022  3.192922e+08   
1 -3.192820e+08  OTHERS-P SETT DT01.0             1  09-

# Merged on TRAN_DATE,TRANAMOUNT & TRAN_ID 

In [22]:
start = time.time()
result5 = pd.merge(dataf1,dataf2, on= ['TRAN_DATE', 'TRANAMOUNT', 'TRAN_ID'], how = 'inner')
end = time.time()
print (result5,'\n',(end-start),'sec')

Empty DataFrame
Columns: [Unnamed: 0_x, TRAN_DATE, TRAN_ID, TRAN_REMARKS_x, ACCOUNTNO_x, TRANAMOUNT, REFERENCENO_x, Unnamed: 0_y, ACCOUNTNO_y, REFERENCENO_y, TRAN_REMARKS_y]
Index: [] 
 0.008973836898803711 sec


In [69]:
start = time.time()
result5 = pd.concat([dataf1,dataf2], join="inner")
end = time.time()
print (result5.head(10),'\n',(end-start),'sec')

   Unnamed: 0 TRAN_DATE    TRAN_ID               TRAN_REMARKS     ACCOUNTNO  \
0           0  9/2/2022  S85305249                        NaN  0103SLMPSWDL   
1           1  9/2/2022  S85305249                        NaN  0103SLMPSWDL   
2           2  9/2/2022  S88796439  224500032693//10101170199  0103SLMPSWDL   
3           3  9/2/2022  S80939874                        NaN  0103SLMPSWDL   
4           4  9/2/2022  S80939874                        NaN  0103SLMPSWDL   
5           5  9/2/2022  S80939874                        NaN  0103SLMPSWDL   
6           6  9/2/2022  S85290736                        NaN  0103SLMPSWDL   
7           7  9/2/2022  S99485662                        NaN  0103SLMPSWDL   
8           8  9/2/2022  S88797032  224500032711//10101170199  0103SLMPSWDL   
9           9  9/2/2022  S85321543                        NaN  0103SLMPSWDL   

     TRANAMOUNT           REFERENCENO  
0 -3.192820e+08  OTHERS-P SETT DT01.0  
1 -4.042220e+03  OTHERS-P DIFF DT 31.  
2 -2.77490

In [71]:
start = time.time()
result3 = pd.merge(dataf1,dataf2, on= ['REFERENCENO', 'ACCOUNTNO','TRAN_ID'], how = 'inner')
end = time.time()
print (result3.head(20),'\n',(end-start),'sec')
result3 = dataf1.to_csv('merge_on_ref_acc_tran.csv')

    Unnamed: 0_x TRAN_DATE_x    TRAN_ID TRAN_REMARKS_x     ACCOUNTNO  \
0              3    9/2/2022  S80939874            NaN  0103SLMPSWDL   
1              3    9/2/2022  S80939874            NaN  0103SLMPSWDL   
2              3    9/2/2022  S80939874            NaN  0103SLMPSWDL   
3              4    9/2/2022  S80939874            NaN  0103SLMPSWDL   
4              4    9/2/2022  S80939874            NaN  0103SLMPSWDL   
5              4    9/2/2022  S80939874            NaN  0103SLMPSWDL   
6              5    9/2/2022  S80939874            NaN  0103SLMPSWDL   
7              5    9/2/2022  S80939874            NaN  0103SLMPSWDL   
8              5    9/2/2022  S80939874            NaN  0103SLMPSWDL   
9             11    9/2/2022  S85321543            NaN  0103SLMPSWDL   
10            13    9/2/2022  S85280052            NaN  0103SLMPSWDL   
11            13    9/2/2022  S85280052            NaN  0103SLMPSWDL   
12            14    9/2/2022  S85280052            NaN  0103SLMP

PermissionError: [Errno 13] Permission denied: 'merge_on_ref_acc_tran.csv'

In [64]:
start = time.time()
data_merge1 = dataf1.merge(dataf2, how = 'inner', left_on = "TRAN_ID", right_on = "TRAN_DATE", validate= 'm:m', sort= True, indicator=True)
end = time.time()
print (data_merge1.head(10),'\n',(end-start),'sec')

Empty DataFrame
Columns: [Unnamed: 0_x, TRAN_DATE_x, TRAN_ID_x, TRAN_REMARKS_x, ACCOUNTNO_x, TRANAMOUNT_x, REFERENCENO_x, Unnamed: 0_y, TRAN_DATE_y, TRANAMOUNT_y, ACCOUNTNO_y, REFERENCENO_y, TRAN_REMARKS_y, TRAN_ID_y, _merge]
Index: [] 
 0.09207630157470703 sec
